In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [4]:
from google.colab import files
uploaded = files.upload()

Saving heart_missing.csv to heart_missing.csv


In [5]:
df = pd.read_csv("heart_missing.csv")

In [6]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52.0,1.0,0.0,NaN,212.0,0.0,1.0,168.0,0.0,1.0,2.0,2.0,3.0,0.0
1,53.0,1.0,0.0,140.0,203.0,1.0,0.0,155.0,1.0,3.1,0.0,0.0,3.0,0.0
2,70.0,1.0,0.0,145.0,174.0,0.0,1.0,125.0,1.0,2.6,0.0,0.0,3.0,0.0
3,61.0,1.0,0.0,148.0,203.0,0.0,1.0,161.0,0.0,0.0,2.0,1.0,3.0,0.0
4,62.0,0.0,0.0,138.0,294.0,1.0,1.0,106.0,0.0,1.9,1.0,3.0,2.0,0.0


In [7]:
df.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
995,44.0,1.0,1.0,120.0,NaN,0.0,1.0,173.0,0.0,NaN,2.0,0.0,3.0,1.0
996,NaN,0.0,0.0,134.0,409.0,0.0,0.0,150.0,1.0,1.9,1.0,NaN,3.0,0.0
997,54.0,1.0,0.0,120.0,188.0,0.0,1.0,113.0,0.0,1.4,1.0,1.0,3.0,0.0
998,42.0,1.0,NaN,136.0,315.0,NaN,1.0,125.0,1.0,1.8,1.0,0.0,1.0,0.0
999,67.0,1.0,0.0,125.0,254.0,1.0,1.0,163.0,NaN,0.2,1.0,2.0,3.0,0.0


In [9]:
df.isnull().sum()

,0
age,100
sex,100
cp,100
trestbps,100
chol,100
fbs,100
restecg,100
thalach,100
exang,100
oldpeak,100


In [10]:
df["age"].fillna(df["age"].mean(), inplace=True)  # เติมค่าด้วยค่าเฉลี่ย
df["chol"].fillna(df["chol"].median(), inplace=True)  # เติมค่าด้วยค่ามัธยฐาน

df["sex"].fillna(df["sex"].mode()[0], inplace=True)
df["thal"].fillna(df["thal"].mode()[0], inplace=True)

df.interpolate(method="linear", inplace=True)
df.dropna(inplace=True)
df.dropna(axis=1, thresh=int(0.5 * len(df)), inplace=True)

df.isnull().sum()

<ipython-input-10-c1175fdce245>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["age"].fillna(df["age"].mean(), inplace=True)  # เติมค่าด้วยค่าเฉลี่ย
<ipython-input-10-c1175fdce245>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [11]:
df["target"] = df["target"].astype(int)
df["target"] = df["target"].apply(lambda x: 1 if x > 0.5 else 0)

In [12]:
Q1 = df["chol"].quantile(0.25)
Q3 = df["chol"].quantile(0.75)
IQR = Q3 - Q1

# ลบข้อมูลที่อยู่นอกช่วง 1.5 เท่าของ IQR
df = df[~((df["chol"] < (Q1 - 1.5 * IQR)) | (df["chol"] > (Q3 + 1.5 * IQR)))]

In [13]:
# แยก Features และ Target
X = df.drop(columns=["target"])
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
# ตั้งค่า K-Fold Cross Validation (k=5)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [16]:
# เทรนโมเดล SVM และ RandomForest
svm_model = SVC(kernel="linear", random_state=42)
svm_model.fit(X_train_scaled, y_train)
y_pred_svm = svm_model.predict(X_test_scaled)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

In [17]:
# คำนวณคะแนน Cross Validation
svm_scores = cross_val_score(svm_model, X_train_scaled, y_train, cv=kf, scoring="accuracy")
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=kf, scoring="accuracy")


In [18]:
acc_svc = accuracy_score(y_test, y_pred_svm)
acc_rf = accuracy_score(y_test, y_pred_rf)

acc_svc, acc_rf
print(f"SVC Accuracy: {acc_svc:.2f}")
print(f"Random Forest Accuracy: {acc_rf:.2f}")

SVC Accuracy: 0.82
Random Forest Accuracy: 0.89


In [30]:
import pickle
from IPython.display import FileLink
# บันทึก SVM Model
with open("svm_model.pkl", "wb") as file:
    pickle.dump(svm_model, file)

# บันทึก Random Forest Model
with open("rf_model.pkl", "wb") as file:
    pickle.dump(rf_model, file)

file.download("svm_model.pkl")
file.download("rf_model.pkl")

AttributeError: '_io.BufferedWriter' object has no attribute 'download'